In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

** to build a predictive model and find out the sales of each product at a particular store **

In [54]:
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")

In [3]:
data = pd.concat([train,test], ignore_index=True)

In [4]:
data.head()

,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type
0,Low Fat,FDA15,249.8092,3735.1380,Dairy,0.016047,9.30,1999,OUT049,Tier 1,Medium,Supermarket Type1
1,Regular,DRC01,48.2692,443.4228,Soft Drinks,0.019278,5.92,2009,OUT018,Tier 3,Medium,Supermarket Type2
2,Low Fat,FDN15,141.6180,2097.2700,Meat,0.016760,17.50,1999,OUT049,Tier 1,Medium,Supermarket Type1
3,Regular,FDX07,182.0950,732.3800,Fruits and Vegetables,0.000000,19.20,1998,OUT010,Tier 3,NaN,Grocery Store
4,Low Fat,NCD19,53.8614,994.7052,Household,0.000000,8.93,1987,OUT013,Tier 3,High,Supermarket Type1


In [5]:
data.isnull().sum()

Item_Fat_Content                0
Item_Identifier                 0
Item_MRP                        0
Item_Outlet_Sales            5681
Item_Type                       0
Item_Visibility                 0
Item_Weight                  2439
Outlet_Establishment_Year       0
Outlet_Identifier               0
Outlet_Location_Type            0
Outlet_Size                  4016
Outlet_Type                     0
dtype: int64

In [6]:
data.describe()

,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Establishment_Year
count,14204.000000,8523.000000,14204.000000,11765.000000,14204.000000
mean,141.004977,2181.288914,0.065953,12.792854,1997.830681
std,62.086938,1706.499616,0.051459,4.652502,8.371664
min,31.290000,33.290000,0.000000,4.555000,1985.000000
25%,94.012000,834.247400,0.027036,8.710000,1987.000000
50%,142.247000,1794.331000,0.054021,12.600000,1999.000000
75%,185.855600,3101.296400,0.094037,16.750000,2004.000000
max,266.888400,13086.964800,0.328391,21.350000,2009.000000


In [7]:
data.apply(lambda x: len(x.unique()))

Item_Fat_Content                 5
Item_Identifier               1559
Item_MRP                      8052
Item_Outlet_Sales             3494
Item_Type                       16
Item_Visibility              13006
Item_Weight                    416
Outlet_Establishment_Year        9
Outlet_Identifier               10
Outlet_Location_Type             3
Outlet_Size                      4
Outlet_Type                      4
dtype: int64

In [8]:
data.dtypes

Item_Fat_Content              object
Item_Identifier               object
Item_MRP                     float64
Item_Outlet_Sales            float64
Item_Type                     object
Item_Visibility              float64
Item_Weight                  float64
Outlet_Establishment_Year      int64
Outlet_Identifier             object
Outlet_Location_Type          object
Outlet_Size                   object
Outlet_Type                   object
dtype: object

In [9]:
categorical_col = [x for x in data.dtypes.index if data.dtypes[x] == 'object']
categorical_col = [x for x in categorical_col if x not in ['Item_Identifier', 'Outlet_Identifier']]
categorical_col

['Item_Fat_Content',
 'Item_Type',
 'Outlet_Location_Type',
 'Outlet_Size',
 'Outlet_Type']

In [10]:
for i in categorical_col:
    print("frequency values in ", i , " = ")
    print(data[i].value_counts())

frequency values in  Item_Fat_Content  = 
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64
frequency values in  Item_Type  = 
Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64
frequency values in  Outlet_Location_Type  = 
Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64
frequency values in  Outlet_Size  = 
Medium    4655
Small     3980
High      1553
Name: Outlet_Size, dtype: int64
frequency values in  Outlet_Type  = 
Supermarket T

treatement for missing values

In [11]:
item_avg_wt = data.pivot_table(values='Item_Weight', index='Item_Identifier')

In [12]:
item_avg_wt.head()

,Item_Weight
Item_Identifier,
DRA12,11.600
DRA24,19.350
DRA59,8.270
DRB01,7.390
DRB13,6.115


In [13]:
item_avg_wt.loc['DRA12']

Item_Weight    11.6
Name: DRA12, dtype: float64

In [14]:
miss_bool = data['Item_Weight'].isnull()
print("missing values = ", sum(miss_bool))
data.loc[miss_bool, 'Item_Weight'] = data.loc[miss_bool,'Item_Identifier'].apply(lambda x: item_avg_wt.loc[x])
print("missing values after imputing = ", sum(data['Item_Weight'].isnull()))

missing values =  2439
missing values after imputing =  0


In [15]:
outlet_size_mode = data.pivot_table(values="Outlet_Size", index='Outlet_Type', aggfunc=(lambda x: x.mode()[0]))

In [16]:
miss_bool_os = data.Outlet_Size.isnull()
data.loc[miss_bool_os, 'Outlet_Size'] = data.loc[miss_bool_os, 'Outlet_Type'].apply(lambda x: outlet_size_mode.loc[x])

In [17]:
item_ol_sale = data.pivot_table(values='Item_Outlet_Sales', index='Item_Type')

In [18]:
miss_bool_ios = data.Item_Outlet_Sales.isnull()

In [19]:
data.loc[miss_bool_ios, 'Item_Outlet_Sales'] = data.loc[miss_bool_ios, 'Item_Type'].apply(lambda x: item_ol_sale.loc[x])

In [20]:
data.isnull().sum()

Item_Fat_Content             0
Item_Identifier              0
Item_MRP                     0
Item_Outlet_Sales            0
Item_Type                    0
Item_Visibility              0
Item_Weight                  0
Outlet_Establishment_Year    0
Outlet_Identifier            0
Outlet_Location_Type         0
Outlet_Size                  0
Outlet_Type                  0
dtype: int64

feature engineering section

In [21]:
item_visibility_avg = data.pivot_table(values='Item_Visibility', index='Item_Identifier')

In [22]:
item_visibility_avg.isnull().sum()

Item_Visibility    0
dtype: int64

In [23]:
miss_item_visibility = data.Item_Visibility==0.0
    

In [24]:
data.loc[miss_item_visibility, 'Item_Visibility'] = data.loc[miss_item_visibility, 'Item_Identifier'].apply(lambda x: item_visibility_avg.loc[x])

finding visibility of perticular product in that shop 

In [25]:
# data[['Item_Outlet_Sales', 'Outlet_Identifier']]
data.pivot_table(values='Item_Visibility', columns='Item_Type', index='Outlet_Identifier', aggfunc=(lambda x: x.max()))

Item_Type,Baking Goods,Breads,Breakfast,Canned,Dairy,Frozen Foods,Fruits and Vegetables,Hard Drinks,Health and Hygiene,Household,Meat,Others,Seafood,Snack Foods,Soft Drinks,Starchy Foods
Outlet_Identifier,,,,,,,,,,,,,,,,
OUT010,0.311090,0.281510,0.239530,0.313935,0.297884,0.281956,0.291865,0.285078,0.255348,0.309390,0.299739,0.305725,0.234733,0.291827,0.293048,0.238366
OUT013,0.185705,0.168046,0.156701,0.195679,0.183142,0.168313,0.183250,0.170177,0.175039,0.185913,0.178928,0.182502,0.146178,0.181899,0.183355,0.153861
OUT017,0.186911,0.169138,0.157719,0.188620,0.184332,0.169406,0.184440,0.171282,0.176175,0.185889,0.180090,0.183687,0.141034,0.175336,0.184545,0.154860
OUT018,0.186616,0.168871,0.157471,0.188323,0.184042,0.169139,0.184150,0.171012,0.175898,0.186826,0.179807,0.183398,0.146896,0.175061,0.184255,0.154617
OUT019,0.304859,0.294473,0.274592,0.328391,0.304737,0.294939,0.321115,0.298205,0.267107,0.325781,0.313542,0.210596,0.256152,0.305265,0.306543,0.249343
OUT027,0.175362,0.167372,0.156072,0.186650,0.182407,0.167637,0.182515,0.169493,0.174336,0.185167,0.178210,0.181769,0.145591,0.173506,0.182619,0.153244
OUT035,0.185824,0.168155,0.156802,0.187523,0.183260,0.168421,0.183368,0.170286,0.175151,0.186033,0.179044,0.182619,0.146272,0.174317,0.183473,0.153960
OUT045,0.186236,0.168527,0.157150,0.187939,0.183667,0.168794,0.183775,0.170664,0.175540,0.186445,0.186779,0.183024,0.141740,0.174704,0.183879,0.154302
OUT046,0.176216,0.168186,0.156832,0.187559,0.183295,0.168453,0.183403,0.170318,0.175184,0.186068,0.179077,0.182654,0.146300,0.178476,0.183507,0.153989


In [26]:
data['Item_Visibility_MeanRatio']= data.apply(lambda x: x['Item_Visibility']/item_visibility_avg.loc[x['Item_Identifier']], axis=1)

In [27]:
data['Item_Visibility_MeanRatio'].describe()

count    14204.000000
mean         1.061884
std          0.235907
min          0.844563
25%          0.925131
50%          0.999070
75%          1.042007
max          3.010094
Name: Item_Visibility_MeanRatio, dtype: float64

In [28]:
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x: x[0:2])
data.Item_Type_Combined.head()

0    FD
1    DR
2    FD
3    FD
4    NC
Name: Item_Type_Combined, dtype: object

In [29]:
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'Food', 'DR':'Drinks', 'NC':'Non-Consumables'})

In [30]:
data['Item_Type_Combined'].value_counts()

Food               10201
Non-Consumables     2686
Drinks              1317
Name: Item_Type_Combined, dtype: int64

In [31]:
data.pivot_table(values='Item_Outlet_Sales', index='Item_Type_Combined')

,Item_Outlet_Sales
Item_Type_Combined,
Drinks,2031.131530
Food,2210.431243
Non-Consumables,2144.637915


In [32]:
data['Outlet_Years'] = 2018 - data['Outlet_Establishment_Year']

In [33]:
data.Outlet_Years.value_counts()

33    2439
31    1553
14    1550
21    1550
19    1550
16    1548
9     1546
11    1543
20     925
Name: Outlet_Years, dtype: int64

In [34]:
data.Outlet_Years.describe()

count    14204.000000
mean        20.169319
std          8.371664
min          9.000000
25%         14.000000
50%         19.000000
75%         31.000000
max         33.000000
Name: Outlet_Years, dtype: float64

In [35]:
data.Item_Fat_Content.unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [36]:
data.Item_Fat_Content.isnull().sum()

0

In [37]:
print(data.Item_Fat_Content.value_counts())
data['Item_Fat_Content']= data['Item_Fat_Content'].replace({'Low Fat':'LF', 'Regular':'reg', 'low fat':'LF', 'LF':'LF', 'reg':'reg'})

Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64


In [38]:
data.Item_Fat_Content.unique()

array(['LF', 'reg'], dtype=object)

In [39]:
data.Item_Fat_Content.isnull().sum()

0

In [40]:
data.Item_Fat_Content.value_counts()

LF     9185
reg    5019
Name: Item_Fat_Content, dtype: int64

In [41]:
data.loc[data['Item_Type_Combined']=='Non-Consumables', 'Item_Fat_Content']="Non-Edible"

In [42]:
data.Item_Fat_Content.value_counts()

LF            6499
reg           5019
Non-Edible    2686
Name: Item_Fat_Content, dtype: int64

In [43]:
data.head()

,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type,Item_Visibility_MeanRatio,Item_Type_Combined,Outlet_Years
0,LF,FDA15,249.8092,3735.1380,Dairy,0.016047,9.30,1999,OUT049,Tier 1,Medium,Supermarket Type1,0.931078,Food,19
1,reg,DRC01,48.2692,443.4228,Soft Drinks,0.019278,5.92,2009,OUT018,Tier 3,Medium,Supermarket Type2,0.933420,Drinks,9
2,LF,FDN15,141.6180,2097.2700,Meat,0.016760,17.50,1999,OUT049,Tier 1,Medium,Supermarket Type1,0.960069,Food,19
3,reg,FDX07,182.0950,732.3800,Fruits and Vegetables,0.017834,19.20,1998,OUT010,Tier 3,Small,Grocery Store,1.000000,Food,20
4,Non-Edible,NCD19,53.8614,994.7052,Household,0.009780,8.93,1987,OUT013,Tier 3,High,Supermarket Type1,1.000000,Non-Consumables,31


Feature Engineering

In [44]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Outlet'] = le.fit_transform(data['Outlet_Identifier'])

In [45]:
var_mod = ['Item_Fat_Content', 'Outlet_Location_Type', 'Outlet_Size', 'Outlet_Type', 'Item_Type_Combined','Outlet_Type','Outlet']
le = LabelEncoder()
for i in var_mod:
    print(i)
    data[i] = le.fit_transform(data[i])

Item_Fat_Content
Outlet_Location_Type
Outlet_Size
Outlet_Type
Item_Type_Combined
Outlet_Type
Outlet


In [46]:
data = pd.get_dummies(data, columns= ['Item_Fat_Content', 'Outlet_Location_Type', 'Outlet_Size', 'Outlet_Type', 'Item_Type_Combined','Outlet_Type','Outlet'])
data.dtypes

Item_Identifier               object
Item_MRP                     float64
Item_Outlet_Sales            float64
Item_Type                     object
Item_Visibility              float64
Item_Weight                  float64
Outlet_Establishment_Year      int64
Outlet_Identifier             object
Item_Visibility_MeanRatio    float64
Outlet_Years                   int64
Item_Fat_Content_0             uint8
Item_Fat_Content_1             uint8
Item_Fat_Content_2             uint8
Outlet_Location_Type_0         uint8
Outlet_Location_Type_1         uint8
Outlet_Location_Type_2         uint8
Outlet_Size_0                  uint8
Outlet_Size_1                  uint8
Outlet_Size_2                  uint8
Outlet_Type_0                  uint8
Outlet_Type_1                  uint8
Outlet_Type_2                  uint8
Outlet_Type_3                  uint8
Item_Type_Combined_0           uint8
Item_Type_Combined_1           uint8
Item_Type_Combined_2           uint8
Outlet_Type_0                  uint8
O

In [47]:
data =data.drop(['Item_Type','Outlet_Establishment_Year'], axis=1)

Model Creation

In [51]:
from sklearn.model_selection import train_test_split

In [60]:
x = data.drop(['Item_Outlet_Sales'], axis=1)

In [62]:
x.head()

,Item_Identifier,Item_MRP,Item_Visibility,Item_Weight,Outlet_Identifier,Item_Visibility_MeanRatio,Outlet_Years,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,...,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,FDA15,249.8092,0.016047,9.30,OUT049,0.931078,19,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,DRC01,48.2692,0.019278,5.92,OUT018,0.933420,9,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,FDN15,141.6180,0.016760,17.50,OUT049,0.960069,19,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,FDX07,182.0950,0.017834,19.20,OUT010,1.000000,20,0,0,1,...,1,0,0,0,0,0,0,0,0,0
4,NCD19,53.8614,0.009780,8.93,OUT013,1.000000,31,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [114]:
# outcome_label = loan_df[outcome_name]
outcome_name = 'Item_Outlet_Sales'
y = data[outcome_name]
y.head()

0    3735.1380
1     443.4228
2    2097.2700
3     732.3800
4     994.7052
Name: Item_Outlet_Sales, dtype: float64

In [77]:
IDcol = ['Item_Identifier', 'Outlet_Identifier']

In [115]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [113]:
X_train.head()

,Item_Identifier,Item_MRP,Item_Visibility,Item_Weight,Outlet_Identifier,Item_Visibility_MeanRatio,Outlet_Years,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,...,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
9770,FDB41,47.8718,0.097232,19.000,OUT013,0.874001,31,0,0,1,...,0,1,0,0,0,0,0,0,0,0
5456,FDY59,93.3462,0.031251,8.195,OUT027,1.032182,33,1,0,0,...,0,0,0,0,0,1,0,0,0,0
10688,FDS39,141.6812,0.022552,6.895,OUT018,0.933420,9,1,0,0,...,0,0,0,1,0,0,0,0,0,0
8749,FDX35,226.4036,0.080236,5.035,OUT018,0.962483,9,0,0,1,...,0,0,0,1,0,0,0,0,0,0
9246,FDW27,157.0314,0.150825,5.860,OUT035,1.028064,14,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [66]:
X_train.to_csv("train_modified.csv", index=False)
X_test.to_csv("test_modified.csv", index=False)

In [68]:
mean_sales = train['Item_Outlet_Sales'].mean()
base1 = test[['Item_Identifier','Outlet_Identifier']]
base1['Item_Outlet_Sales'] = mean_sales
base1.to_csv("alg0.csv",index=False)

/home/priyankat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [129]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [70]:
alg1 = LinearRegression()

In [74]:
np.array(y_train)

array([2132.86774369, 1018.0082    , 2158.97791059, ..., 6145.334     ,
       1649.8524    ,  965.41      ])

In [82]:
predictors = [x for x in X_train.columns if x not in IDcol]

In [88]:
X_train[predictors].head()

,Item_MRP,Item_Visibility,Item_Weight,Item_Visibility_MeanRatio,Outlet_Years,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,Outlet_Location_Type_0,Outlet_Location_Type_1,...,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
9770,47.8718,0.097232,19.000,0.874001,31,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
5456,93.3462,0.031251,8.195,1.032182,33,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10688,141.6812,0.022552,6.895,0.933420,9,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8749,226.4036,0.080236,5.035,0.962483,9,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
9246,157.0314,0.150825,5.860,1.028064,14,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,0


In [89]:
model = alg1.fit(X_train[predictors], y_train)
model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [93]:
# coef1 = pd.Series(alg1.coef_, predictors).sort_values()
# coef1.plot(kind='bar', title='Model Coefficients')

In [95]:
alg1_predictions = model.predict(X_train[predictors])

In [124]:
from sklearn import cross_validation, metrics

In [119]:
# cv_score = cross_validation.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=20, scoring='mean_squared_error')
cv_score = cross_validation.cross_val_score(alg1, X_train[predictors], y_train, cv=20, scoring='mean_squared_error')

/home/priyankat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/priyankat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/priyankat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/priyankat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sam

In [120]:
cv_score

array([-1358367.79094209, -1211660.63933907, -1250969.66489234,
        -883059.13786052, -1073165.52298809, -1381889.07805588,
       -1231639.50752922, -1027267.48497921, -1031713.19626749,
       -1175109.09319002, -1022628.74155936, -1264030.1749544 ,
       -1290651.78879448, -1055199.65024124, -1228296.89740563,
       -1045913.24584833, -1196176.82141762, -1044822.84403473,
       -1276000.41305275, -1272968.59374336])

In [121]:
cv_score = np.sqrt(np.abs(cv_score))
cv_score

array([1165.49036501, 1100.75457725, 1118.46755201,  939.71226333,
       1035.93702656, 1175.53778249, 1109.79255157, 1013.54204894,
       1015.73283705, 1084.0244892 , 1011.25107741, 1124.29096543,
       1136.06856694, 1027.22911283, 1108.28556672, 1022.69900061,
       1093.69868859, 1022.16576153, 1129.60188255, 1128.25909868])

In [126]:
np.sqrt(metrics.mean_squared_error(y_train.values, alg1_predictions))

1078.0487185226345

Ridge Regression model

In [131]:
alg2 = Ridge()
model2 = alg2.fit(X_train[predictors], y_train)
print(model2)
alg2_predictions = model2.predict(X_train[predictors])
# cv_score = cross_validation.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=20, scoring='mean_squared_error')
cv_score2 = cross_validation.cross_val_score(alg2, X_train[predictors], y_train, cv=20, scoring='mean_squared_error')
print(cv_score2)
print(np.sqrt(metrics.mean_squared_error(y_train.values, alg2_predictions)))

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)


/home/priyankat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/priyankat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/priyankat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/priyankat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sam

[-1358339.55171894 -1211680.59620996 -1250833.38069638  -883081.02199374
 -1073130.5349034  -1381878.91369854 -1231521.10070615 -1027182.22072718
 -1031726.20762643 -1175072.09710996 -1022643.07428154 -1264079.26002507
 -1290645.93742423 -1055247.17298185 -1228292.20852398 -1045844.14528033
 -1196216.48392485 -1044853.45747895 -1275995.21979543 -1272979.11362611]
1078.048798495958


/home/priyankat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/priyankat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/priyankat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/priyankat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sam

In [132]:
from sklearn.tree import DecisionTreeRegressor

In [133]:
alg3 = DecisionTreeRegressor()
model3 = alg3.fit(X_train[predictors], y_train)
print(model3)
alg3_predictions = model3.predict(X_train[predictors])
# cv_score = cross_validation.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=20, scoring='mean_squared_error')
cv_score3 = cross_validation.cross_val_score(alg3, X_train[predictors], y_train, cv=20, scoring='mean_squared_error')
print(cv_score3)
print(np.sqrt(metrics.mean_squared_error(y_train.values, alg3_predictions)))

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')


/home/priyankat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/priyankat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/priyankat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/priyankat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sam

[-2522414.089173   -2741307.64317841 -2084573.84893844 -2014980.38854933
 -2428983.28962608 -2532641.67835068 -2761750.95312681 -2510917.70021174
 -1996643.94023205 -2438174.94210333 -2114558.44040958 -2410134.90209036
 -2570922.97016538 -2048788.94282717 -2256253.91595457 -2244746.40525941
 -2311591.60553518 -2190178.78750004 -3063093.47091145 -2540369.79286496]
0.0


/home/priyankat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/priyankat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
